In [11]:
import pandas as pd

Загрузим табличку с нашими данными про длительность поездок такси! 

In [12]:
df = pd.read_csv('taxi_dataset_with_target.csv', index_col=0)

df.head()

,vendor_id,pickup_datetime,passenger_count,store_and_fwd_flag,trip_duration,distance_km
id,,,,,,
id2875421,1,2016-03-14 17:24:55,930.399753,0,455.0,1.500479
id2377394,0,2016-06-12 00:43:35,930.399753,0,663.0,1.807119
id3858529,1,2016-01-19 11:35:24,930.399753,0,2124.0,6.392080
id3504673,1,2016-04-06 19:32:31,930.399753,0,429.0,1.487155
id2181028,1,2016-03-26 13:30:55,930.399753,0,435.0,1.189925


Попробуем построить модель Линейной Регрессии!

В качестве Выборки используйте таргетную колонку (*trip_duration*) и все признаки, кроме времени начала заказа (*pikcup_datetime*)

Для начала воспользуемся "коробочным решением". Для этого создайте переменную model, положите в нее класс **LinearRegression** из модуля **linear_model** библиотеки **sklearn**.

Далее воспользуйтесь методом **fit**, положите в него через запятую: массив **X**, состоящий из признаков объектов (можно pandas dataframe'ом, а можно numpy-массивом), и массив **Y** с таргетами.

In [13]:
from sklearn.linear_model import LinearRegression

X = df.drop(['pickup_datetime', 'trip_duration'], axis = 1)
Y = df['trip_duration']

model = LinearRegression()
model.fit(X, Y)

LinearRegression()

Чтобы посмотреть на значения полученных коэффициентов модели, следует обратиться к атрибуту **coef_** класса линейной регрессии. Для просмотра свободного веса, к атрибуту **intercept_** 

In [14]:
import numpy as np
print(np.around(model.coef_, decimals = 3))
print(np.around(model.intercept_, decimals = 3))


[198.463   0.296  56.469 115.274]
171.657


Теперь реализуйте функцию LinearRegressionByMatrix, которая будет принимать на вход 3 параметра:

Матрицу объект-признак **(X)**, вектор с ответами **(Y)**, булевый параметр **fit_intercept**, цель которого - добавить константный признак (состоящий из единичек), если True, и ничего не делать, если False.

Функция должна вернуть одномерный np.array объект с оцененными **$\beta_1, ..., \beta_n, \beta_0$**

Реализуйте решение через аналитическое решение в матрицах из лекции и практики:

$$
\beta^* = (X^T \cdot X)^{-1} \cdot X^T \cdot Y
$$

In [15]:
import numpy as np

def LinearRegressionByMatrix(X: np.array, Y: np.array, fit_intercept: bool = True):
        """
        :param X: матрица объектов
        :param Y: вектор (матрица с 1 столбцом) ответов
        :param fit_intercept: добавить ли константную переменную в данные?
        
        :return: одномерный numpy-array с полученными коэффициентами beta
        """
        if fit_intercept == True:
            X = np.concatenate([X, np.ones((X.shape[0],1),dtype=X.dtype)], axis=1)
        else:
            pass
        xxt = np.dot(X.T, X)
        xxt_inv = np.linalg.inv(xxt)
        xxt_inv_xxt = np.dot(xxt_inv, X.T)
        final_betas = np.dot(xxt_inv_xxt, Y)
        return final_betas

In [16]:
X_train = df.drop(['trip_duration', 'pickup_datetime'], axis=1).values
Y_train = df['trip_duration'].values

betas = LinearRegressionByMatrix(X_train, Y_train)
print(np.around(betas, decimals = 3))

[198.463   0.296  56.469 115.274 171.657]


Получились ли коэффициенты такими же, как и в "коробочном" варианте?

Имея коэффициенты модели, можно восстановить предсказания для каждого объекта!

Сделайте это через операцию матричного произведения матрицы **X** и полученных коэффициентов (как результат работы *LinearRegressionByMatrix*)

In [17]:
X_ = X.to_numpy()
X__ = np.concatenate([X_, np.ones((X_.shape[0],1),dtype=X_.dtype)], axis=1)
Y_predict = np.dot(X__, betas)
Y_predict

array([ 818.7747282 ,  655.65912268, 1382.6469154 , ..., 1548.74134353,
        573.4306718 ,  578.2338068 ])